In [10]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Conv2D, Activation, BatchNormalization, \
    MaxPooling2D, Flatten, Dropout
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

In [11]:
import tensorflow


In [12]:
def load_dataset():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    return X_train, y_train, X_test, y_test

In [13]:
def data_split():
    X_train, y_train, X_test, y_test = load_dataset()
    (X_train, X_valid) = X_train[5000:], X_train[:5000]
    (y_train, y_valid) = y_train[5000:], y_train[:5000]
    return X_train, X_test, X_valid, y_train, y_test, y_valid

In [14]:
def preprocessing():
    X_train, X_test, X_valid, y_train, y_test, y_valid = data_split()
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / (std + 1e-7)
    X_valid = (X_valid - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7)
    num_classes = len(np.unique(y_train))
    y_train = to_categorical(y_train, num_classes=num_classes)
    y_test = to_categorical(y_test, num_classes=num_classes)
    y_valid = to_categorical(y_valid, num_classes=num_classes)
    return X_train, X_test, X_valid, y_train, y_test, y_valid

In [15]:
def augmentation():
    X_train, X_test, X_valid, y_train, y_test, y_valid = preprocessing()
    datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1,
                                 height_shift_range=0.1, horizontal_flip=True,
                                 vertical_flip=False)
    datagen.fit(X_train)
    return datagen, X_train, X_test, X_valid, y_train, y_test, y_valid

In [16]:
def model_creation():
    datagen, X_train, X_test, X_valid, y_train, y_test, y_valid = augmentation()
    base_hidden_layer = 32
    weight_decay = 1e-4
    model = Sequential()

    # CONV-1
    model.add(Conv2D(filters=base_hidden_layer, kernel_size=3, padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay),
                     input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    # CONV-2
    model.add(Conv2D(filters=base_hidden_layer, kernel_size=3, padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    # POOL + Dropout
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # CONV-3
    model.add(Conv2D(filters=base_hidden_layer * 2, kernel_size=3, padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay),
                     input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    # CONV-4
    model.add(Conv2D(filters=base_hidden_layer * 2, kernel_size=3, padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    # POOL + Dropout
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # CONV-5
    model.add(Conv2D(filters=base_hidden_layer * 4, kernel_size=3, padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay),
                     input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    # CONV-6
    model.add(Conv2D(filters=base_hidden_layer * 4, kernel_size=3, padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    # POOL + Dropout
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    # FC-7
    model.add(Flatten())
    model.add(Dense(10., activation='softmax'))

    return model, datagen, X_train, X_test, X_valid, y_train, y_test, y_valid


In [17]:
def train_model():
    model, datagen, X_train, X_test, X_valid, y_train, y_test, y_valid = model_creation()
    batch_size = 128
    epochs = 10
    # checkpointer = ModelCheckpoint(filepath='model.CIFAR_PART2.hdf5', verbose=2,
    #                                save_best_only=True)
    optimizer = optimizers.Adam(lr=0.001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                  metrics=['accuracy'])
    history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=epochs, verbose=2, validation_data=(X_valid, y_valid))
    return history, model, X_train, X_test, X_valid, y_train, y_test, y_valid

In [18]:
def predict():

    history, model, X_train, X_test, X_valid, y_train, y_test, y_valid = train_model()

    score = model.evaluate(X_test, y_test, verbose=1, batch_size=128)
    print(score[1] * 100)


predict()

Epoch 1/10
351/351 - 21s - loss: 2.0146 - accuracy: 0.3878 - val_loss: 1.8708 - val_accuracy: 0.3804 - 21s/epoch - 59ms/step
Epoch 2/10
351/351 - 20s - loss: 1.4952 - accuracy: 0.5164 - val_loss: 1.3396 - val_accuracy: 0.5694 - 20s/epoch - 56ms/step
Epoch 3/10
351/351 - 19s - loss: 1.2501 - accuracy: 0.5939 - val_loss: 1.2066 - val_accuracy: 0.6176 - 19s/epoch - 55ms/step
Epoch 4/10
351/351 - 20s - loss: 1.1412 - accuracy: 0.6298 - val_loss: 0.9969 - val_accuracy: 0.6808 - 20s/epoch - 58ms/step
Epoch 5/10
351/351 - 19s - loss: 1.0379 - accuracy: 0.6619 - val_loss: 0.8906 - val_accuracy: 0.7156 - 19s/epoch - 55ms/step
Epoch 6/10
351/351 - 19s - loss: 0.9426 - accuracy: 0.6925 - val_loss: 0.8650 - val_accuracy: 0.7272 - 19s/epoch - 55ms/step
Epoch 7/10
351/351 - 19s - loss: 0.9000 - accuracy: 0.7088 - val_loss: 0.8236 - val_accuracy: 0.7522 - 19s/epoch - 55ms/step
Epoch 8/10
351/351 - 19s - loss: 0.8490 - accuracy: 0.7275 - val_loss: 0.7444 - val_accuracy: 0.7618 - 19s/epoch - 55ms/step
